In [1]:
"""
'gorilla-openfunctions-v2-q3_K_L.gguf',
'gorilla-openfunctions-v2-q4_K_M.gguf',

"""
from llama_cpp import Llama
import os
_list = os.listdir("__FUNC_MODELS")


In [11]:
def complext_number_addition(a:int,a_i:int,b:int,b_i:int):
        '''
        used for the addtion of the two complex numbers
        '''
        
        res = f"{int(a)+int(b)}+{int(a_i)+int(b_i)}i"
        # print(f"{a}+{a_i}i + {b}+{b_i}i = ",res)
        return res

def complext_number_multiply(a:int,a_i:int,b:int,b_i:int):
        '''
        used for the addtion of the two complex numbers
        '''
        res = f"{int(a)*int(b)}+{int(a_i)*int(b_i)}i"
        # print(f"{a}+{a_i}i X {b}+{b_i}i = ",res)
        return res


import sqlite3,inspect

def execute_sql(query: str):
    """Runs SQL code for the given schema. Make sure to properly leverage the schema to answer the user's question in the best way possible. 
                        "CREATE TABLE customer_information (
                            id INTEGER PRIMARY KEY AUTOINCREMENT,
                            agent_name TEXT,
                            customer_email TEXT,
                            customer_order TEXT,
                            customer_phone TEXT,
                            customer_sentiment TEXT
                        );
        """
    table_name = "customer_information"

    conn = sqlite3.connect('extracted.db')
    cursor = conn.cursor()

    cursor.execute(query)

    

    results = cursor.fetchall()
#     print("Query operation executed successfully. Number of rows returned:", len(results))

    conn.close()
    return results

In [5]:
from llama_cpp import Llama
import json

llm = Llama(f"__FUNC_MODELS/gorilla-openfunctions-v2-q4_K_M.gguf",verbose=False,n_ctx=2048)
print('')
def get_prompt_gorrila(user_query: str, functions: list = []) -> str:

    system = "You are an AI programming assistant, you only returns the correct function call for user query."
    if len(functions) == 0:
        return f"{system}\n### Instruction: <<question>> {user_query}\n### Response: "
    functions_string = json.dumps(functions)
    return f"{system}\n### Instruction: <<function>>{functions_string}\n<<question>>{user_query}\n### Response: "


def get_prompt(query):
    return """
    <example_response>
    Function:
        def complext_number_addition(a:int,a_i:int,b:int,b_i:int):
            '''
            used for the addtion of the two complex numbers
            '''

        Example:
        User Query: addition of two complex numbers: 4+5i and 3+7i?
        response: {
            "function": "complext_number_addition(a=4,a_i=5,b=3,b_i=7)"
        }

    Function:
        def complext_number_multiply(a:int,a_i:int,b:int,b_i:int):
            '''
            used for the multiplication of the two complex numbers
            '''
        Example:
        User Query: addition of two complex numbers: 4+5i and 3+7i?
        response: {
            "function": "complext_number_multiply(a=4,a_i=5,b=3,b_i=7)"
        }
    
    </example_response>
    CREATE TABLE customer_information (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        agent_name TEXT,
        customer_email TEXT,
        customer_order TEXT,
        customer_phone TEXT,
        customer_sentiment TEXT
    );

    Function:
    execute_sql(sql: str)
        '''Runs SQL code for the given schema. Make sure to properly leverage the schema to answer the user's question in the best way possible. '''
    """\
    f"""\n\n <question>{query}?"""



functions = [
    {
        "name": "complext_number_multiply",
        "description": "muliplication of two complex numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "string",
                    "description": "real value part of first complex number",
                },
                "a_i": {
                    "type": "string",
                    "description": "imaginary value part of first complex number",
                },
                "b": {
                    "type": "string",
                    "description": "real value part of second complex number",
                },
                "b_i": {
                    "type": "string",
                    "description": "real value part of second complex number",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["a","a_i","b","b_i"],
        },
    },
    {
        "name": "complext_number_addition",
        "description": "addition of two complex numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "string",
                    "description": "real value part of first complex number",
                },
                "a_i": {
                    "type": "string",
                    "description": "imaginary value part of first complex number",
                },
                "b": {
                    "type": "string",
                    "description": "real value part of second complex number",
                },
                "b_i": {
                    "type": "string",
                    "description": "real value part of second complex number",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["a","a_i","b","b_i"],
        },
    },
    {
        "name": "execute_sql",
        "description": """Runs SQL code for the given schema. Make sure to properly leverage the schema to answer the user's question in the best way possible. 
                        "CREATE TABLE customer_information (
                            id INTEGER PRIMARY KEY AUTOINCREMENT,
                            agent_name TEXT,
                            customer_email TEXT,
                            customer_order TEXT,
                            customer_phone TEXT,
                            customer_sentiment TEXT
                        );
                    """,
                
        "parameters": {
                "type":"object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Sql query to be executed according to user request ",
                    }
                },
                "required": ["query"],
                
        }
    }
]




In [6]:
_output = []
_input =  [
            #  "how many number of customer  are there in customer_information who has Rachel as agent",
            "can you tell me how many different agents are there, and give name also",
            "can you give customer_phone along with there name of customer who have happy sentiment",
            "can you give all agents their customer email of customer ",
            "can you give all different kinds of sentimets customers are having",
            "in customer_information is customer with email johndoe@email.com is frustrated?",
            "give top 10 rows from customer_information",
            "if i have two numbers what can be the mulitpy of these two 89+5i and 3+7i",
            "multiplication of two complex numbers 298+9i and 213+37i ",
            "what if i add two complex numbers 37+9i and 2313+37i "
            "what if i add two complex numbers 3+11i and 1+7i and multipy with 2+7i with 2+5i "
        ]

for i in _input:
    user_prompt = get_prompt_gorrila(i,functions)
    print(i,end='\r')
    output = llm(user_prompt,
                    max_tokens=512,  
                    stop=["<|EOT|>"], 
                    echo=False        
                )

    _output.append(output["choices"][0]['text'])


In [7]:
# _output = _output[0].split('<<function>>')
_output

[' <<function>>Error, not enough information provided. The user\'s question involves querying a database table for distinct agent names but does not specify the schema of the customer_information table or provide an example query to match against it. Without this context or sample input/output pair, we cannot determine whether all necessary fields and types have been included in the function definition that would allow us to construct the correct SQL statement properly based on user intent for querying distinct agent names from a database using SQLite syntax with appropriate column references within an existing table schema of "customer_information".',
 ' <<function>>execute_sql(query=\'SELECT agent_name,customer_phone FROM customer_information WHERE customer_sentiment="happy"\')',
 ' <<function>>execute_sql(query="SELECT agent_name, customer_email FROM customer_information WHERE id = ")',
 ' <<function>>execute_sql(query="SELECT * FROM customer_information WHERE customer_sentiment = \

In [8]:
for _in,_out in zip(_input,_output):
    try:
        res = [eval(i_out) for i_out in _out.split('<<function>>')[1:]]
        print("query : ",_in,"\nresult is  :",res,"\n\n")
        
    except Exception as e:
        print(e)
        continue


unterminated string literal (detected at line 1) (<string>, line 1)
query :  can you give customer_phone along with there name of customer who have happy sentiment 
result is  : [[('Agent Smith', ''), ('Tom', '+1 123-456-7890'), ('Alex', ''), ('Sarah', ''), ('BrownBox', '123-456-7890'), ('Alex', '123-456-7890'), ('Rachel', ''), ('Sarah', '')]] 


incomplete input
query :  can you give all different kinds of sentimets customers are having 
result is  : [[]] 


query :  in customer_information is customer with email johndoe@email.com is frustrated? 
result is  : [[]] 


query :  give top 10 rows from customer_information 
result is  : [[(1, 'Agent Smith', '', '12346', '', 'happy'), (2, 'Rachel', 'janedoe@email.com', 'BB789012', '', 'frustrated'), (3, 'Tom', 'johndoe@email.com', '', '+1 123-456-7890', 'happy'), (4, 'Alex', '', '789101', '', 'happy'), (5, 'Sarah', 'jane.doe@email.com', '987654', '', 'happy'), (6, 'BrownBox', 'john.doe@gmail.com', 'BB98765432', '123-456-7890', 'happy'), (7,

In [48]:
[eval(i_out) for i_out in _out.split('<<function>>')[1:]]

Query operation executed successfully. Number of rows returned: 1


[[(2,)]]

# CREATE FUNCTION SCHEMA

In [266]:
import inspect
import json

def create_function_schema(func):

    signature = inspect.signature(func)
    docstring = inspect.getdoc(func) or ""
    schema = {
        "name": func.__name__,
        "description": docstring.strip(),
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    }
    
    for param_name, param in signature.parameters.items():
        param_type = "string"  
        if param.annotation != inspect.Parameter.empty:
            if param.annotation == int:
                param_type = "integer"
            elif param.annotation == float:
                param_type = "number"
            elif param.annotation == bool:
                param_type = "boolean"
        
        schema["parameters"]["properties"][param_name] = {
            "type": param_type,
            "description": f"{param_name} parameter"
        }
        
        if param.default == inspect.Parameter.empty:
            schema["parameters"]["required"].append(param_name)
    
    return schema


# schema = create_function_schema(execute_sql)
# print(json.dumps(schema, indent=2))

# LANGCHAIN FUNCTION CALLING WITH AWS 

In [9]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

@tool
def complex_numbers_multiply(a: int, a_i:int, b: int, b_i:int) -> tuple:
    """Multiplies two complex numbers a and b.

    Args:
        a: first int
        a_i: first imaginary int
        b: second int
        b_i: second imaginary int
    """
    return a * b,a_i*b_i

tools = [add, multiply,complex_numbers_multiply]

In [243]:
from langchain.chat_models.bedrock import BedrockChat
from langchain_experimental.llms.anthropic_functions import AnthropicFunctions
import boto3
import botocore

model_name = "anthropic.claude-v2"
region_name = "us-east-1"

client = boto3.client(service_name="bedrock-runtime",region_name=region_name)

In [244]:
# from langchain_core.pydantic_v1 import BaseModel, Field

# class add(BaseModel):
#     """Add two integers together."""

#     a: int = Field(..., description="First integer")
#     b: int = Field(..., description="Second integer")


# class multiply(BaseModel):
#     """Multiply two integers together."""

#     a: int = Field(..., description="First integer")
#     b: int = Field(..., description="Second integer")


# tools = [add, multiply]

In [11]:
from langchain_aws import ChatBedrockConverse
from langchain_core.output_parsers.openai_tools import PydanticToolsParser


llm = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0,
    max_tokens=None,
).bind_tools(tools)

# chain = llm | PydanticToolsParser(tools=[multiply, add])


c:\Users\SIVERMA\Documents\Experimenting\AGENT\AGENTS\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The class `ChatBedrockConverse` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [217]:

output = llm.invoke("what is the result of two complex numbers  22+41i * 324+92i")

In [218]:
_fun_dict = {
    "add":add,
    "multiply":multiply,
    "complex_numbers_multiply":complex_numbers_multiply
    
}

In [219]:
output.tool_calls

[{'name': 'complex_numbers_multiply',
  'args': {'a': 22, 'a_i': 41, 'b': 324, 'b_i': 92},
  'id': 'tooluse_pLOgRFbTR-ecWaJmAH8VTQ'}]

In [220]:
_fun_dict[output.tool_calls[0]['name']].invoke(output.tool_calls[0]['args'])

(7128, 3772)

In [246]:
from langchain_core.messages import HumanMessage, ToolMessage


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage("what is the result when I add 23 + 45 + + 9")]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='what is the result when I add 23 + 45 + + 9'),
 AIMessage(content=[{'type': 'text', 'text': 'To add 23, 45, and 9, we can use the "add" tool like this:'}, {'type': 'tool_use', 'name': 'add', 'input': {'a': 23, 'b': 45}, 'id': 'tooluse_gTJf3XO0Q2GvDrltYdfwRQ'}], response_metadata={'ResponseMetadata': {'RequestId': 'dc2de1af-9bdc-4517-8ab5-b90668d95a75', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 05 Jul 2024 10:19:13 GMT', 'content-type': 'application/json', 'content-length': '340', 'connection': 'keep-alive', 'x-amzn-requestid': 'dc2de1af-9bdc-4517-8ab5-b90668d95a75'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': 1673}}, id='run-f8e3ecd9-0c01-4ffa-9240-ad6c5599a719-0', tool_calls=[{'name': 'add', 'args': {'a': 23, 'b': 45}, 'id': 'tooluse_gTJf3XO0Q2GvDrltYdfwRQ'}], usage_metadata={'input_tokens': 353, 'output_tokens': 92, 'total_tokens': 445}),
 ToolMessage(content='68', tool_call_id='tooluse_gTJf3XO0Q2GvDrltYdfwRQ')]

In [247]:
def process_query(query: str):
    messages = [HumanMessage(query)]
    while True:
        ai_msg = llm_with_tools.invoke(messages)
        messages.append(ai_msg)
        
        if not ai_msg.tool_calls:
            break
        
        for tool_call in ai_msg.tool_calls:
            selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
            tool_output = selected_tool.invoke(tool_call["args"])
            messages.append(ToolMessage(content=str(tool_output), tool_call_id=tool_call["id"]))
    
    return messages

In [255]:
result = process_query("what will be the sum of 4+5 multipy 5")
result

[HumanMessage(content='what will be the sum of 4+5 multipy 5'),
 AIMessage(content=[{'type': 'text', 'text': 'Okay, let\'s break this down step-by-step:\n\n1. To get the sum of 4 + 5, we use the "add" tool:'}, {'type': 'tool_use', 'name': 'add', 'input': {'a': 4, 'b': 5}, 'id': 'tooluse_dztxboQfRASuXnrTjKkmbA'}], response_metadata={'ResponseMetadata': {'RequestId': '80789c9f-7300-43e7-9de3-26e8701209a7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 05 Jul 2024 11:05:11 GMT', 'content-type': 'application/json', 'content-length': '376', 'connection': 'keep-alive', 'x-amzn-requestid': '80789c9f-7300-43e7-9de3-26e8701209a7'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': 3158}}, id='run-b6194400-4d94-4005-863d-ad66c637689f-0', tool_calls=[{'name': 'add', 'args': {'a': 4, 'b': 5}, 'id': 'tooluse_dztxboQfRASuXnrTjKkmbA'}], usage_metadata={'input_tokens': 350, 'output_tokens': 106, 'total_tokens': 456}),
 ToolMessage(content='9', tool_call_id='tooluse_dztxboQ

# COMPLEX MULTI QUERY HANDLE USING `MISTRAL LARGE`

In [12]:
from operator import itemgetter
from typing import Dict, List, Union
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

@tool
def execute_sql(query: str):
    """Runs SQL code for the given schema. Make sure to properly leverage the schema to answer the user's question in the best way possible. 
                        "CREATE TABLE customer_information (
                            id INTEGER PRIMARY KEY AUTOINCREMENT,
                            agent_name TEXT,
                            customer_email TEXT,
                            customer_order TEXT,
                            customer_phone TEXT,
                            customer_sentiment TEXT
                        );
        """
    table_name = "customer_information"

    conn = sqlite3.connect('extracted.db')
    cursor = conn.cursor()

    cursor.execute(query)

    

    results = cursor.fetchall()
    conn.close()
    return results

tools = [multiply, exponentiate, add,execute_sql]
llm = ChatBedrockConverse(
    model="mistral.mistral-large-2402-v1:0",
    temperature=0,
    max_tokens=None,
)

llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}


def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools #| call_tools

c:\Users\SIVERMA\Documents\Experimenting\AGENT\AGENTS\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The class `ChatBedrockConverse` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [ ]:
x = chain.invoke(
    "What's 23 times 7, and what's five times 18 and add a million plus a billion and cube thirty-seven"
)

In [15]:
x.tool_calls

[{'name': 'multiply',
  'args': {'first_int': 23, 'second_int': 7},
  'id': 'tooluse_3IsmunGATV6VmQJ4QyQS0w'},
 {'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 18},
  'id': 'tooluse_tX7vE5cFQB2gtBZHBqsmVw'},
 {'name': 'exponentiate',
  'args': {'base': 37, 'exponent': 3},
  'id': 'tooluse_GCBtVqNeSgOpyfW_uUM5Qw'},
 {'name': 'add',
  'args': {'first_int': 1000000, 'second_int': 1000000000},
  'id': 'tooluse_n7lq3hpmT8q4pRhuXjuu5A'}]

In [237]:
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='what is the result when I add 23 + 45 + + 9'),
 AIMessage(content=[{'type': 'text', 'text': 'To add 23, 45, and 9, we can use the "add" tool like this:'}, {'type': 'tool_use', 'name': 'add', 'input': {'a': 23, 'b': 45}, 'id': 'tooluse_ShthsTbKRQ2AdL-6TYlqRA'}], response_metadata={'ResponseMetadata': {'RequestId': '85fb31d8-0dd6-469b-92e6-f01b6791bb9e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 05 Jul 2024 09:50:30 GMT', 'content-type': 'application/json', 'content-length': '340', 'connection': 'keep-alive', 'x-amzn-requestid': '85fb31d8-0dd6-469b-92e6-f01b6791bb9e'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': 1644}}, id='run-9e2f000d-79ca-4fd9-87fd-0d2438f4d351-0', tool_calls=[{'name': 'add', 'args': {'a': 23, 'b': 45}, 'id': 'tooluse_ShthsTbKRQ2AdL-6TYlqRA'}], usage_metadata={'input_tokens': 353, 'output_tokens': 92, 'total_tokens': 445}),
 ToolMessage(content='68', tool_call_id='tooluse_ShthsTbKRQ2AdL-6TYlqRA')]

In [101]:
from langchain_core.runnables import RunnableLambda

async def reverse(s: str) -> str:
    return s[::-1]

chain = RunnableLambda(func=reverse) | llm

events = [
    event async for event in chain.astream_events("hello", version="v2")
]
events[-1]["data"]['output'].content[0]['text']

'"olleh" is "hello" spelled backwards.'

# WITH LLAMA INDEX

In [ ]:
# %pip install llama-index-llms-bedrock-converse
# %pip install llama-index-embeddings-huggingface
# !pip install llama-index

In [56]:
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.tools import FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [57]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [58]:
llm = BedrockConverse(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    region_name="us-east-1",
)

In [59]:
from llama_index.core.agent import FunctionCallingAgentWorker

agent_worker = FunctionCallingAgentWorker.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = agent_worker.as_agent()

In [104]:
response = agent.chat("what if i add 4 with 9 and multipy the result with 12?")
print(str(response))
print(response.sources)

Added user message to memory: what if i add 4 with 9 and multipy the result with 12?
=== LLM Response ===
Okay, let's do the calculation step-by-step:

1. Add 4 and 9:
   4 + 9 = 13

2. Multiply the result (13) by 12:
   13 * 12 = 156

So, if we add 4 and 9, and then multiply the result by 12, the final answer is 156.
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What is the result of 4 + 9 multiplied by 12?"}
=== Function Output ===
The provided context information does not contain any information about the result of 4 + 9 multiplied by 12. The context appears to be about financial and legal terminology related to a company's financial statements and loan agreements. Without any relevant information in the given context, I cannot determine the result of the arithmetic expression 4 + 9 multiplied by 12.

[ToolOutput(content="The provided context information does not contain any information about the result of 4 + 9 multiplied by 12. The context appears to be

In [68]:
# EXECUTE THIS IN CMD

# !mkdir -p 'data/10k/'
# !curl -o 'data/10k/uber_2021.pdf' 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf'


In [72]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.bedrock_converse import BedrockConverse

embed_model = HuggingFaceEmbedding(model_name=r"C:\Users\SIVERMA\Documents\Experimenting\AGENT\paraphrase-multilingual-MiniLM-L12-v2")
query_llm = BedrockConverse(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    region_name="us-east-1",
)

uber_docs = SimpleDirectoryReader(
    input_files=["./data/10k/uber_2021.pdf"]
).load_data()

uber_index = VectorStoreIndex.from_documents(
    uber_docs, embed_model=embed_model
)
uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm=query_llm)
query_engine_tool = QueryEngineTool(
    query_engine=uber_engine,
    metadata=ToolMetadata(
        name="uber_10k",
        description=(
            "Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)

In [76]:
from llama_index.core.agent import FunctionCallingAgentWorker

agent_worker = FunctionCallingAgentWorker.from_tools(
    [query_engine_tool], llm=llm, verbose=True
)
agent = agent_worker.as_agent()

In [ ]:
response = agent.chat(
    "Tell me both the risk factors and tailwinds for Uber? Do two parallel tool calls."
)

In [78]:

print(str(response))

In summary, the key risk factors for Uber include non-income tax matters, competitive environment, reliance on independent providers, estimation uncertainties, and credit risk concentration. The key tailwinds appear to be the launch of Uber One, the "Super App" integration, growth in advertising merchants, the Cornershop acquisition, and Uber's leading market positions.
